In [67]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [68]:
### importing the data into dataframes
order_data = pd.read_excel('unicommerce_data.xlsx', sheet_name = 'order_data')
category_data = pd.read_excel('unicommerce_data.xlsx', sheet_name  = 'category_data')

In [69]:
order_data.head()

,sku_code,order_count
0,805L,185
1,JR-202-38,32
2,K1786_L,69
3,FDRSS1744XL,6
4,BMAXBLUSQAR,56


In [70]:
category_data.head()

,category_id,category,sku_code
0,4,Fashion & Accessories,538XL
1,4,Fashion & Accessories,JR-LK-SideB-Lux-9017-NBlue-44
2,4,Fashion & Accessories,L766L
3,4,Fashion & Accessories,K1612_L
4,4,Fashion & Accessories,OCRM012TQG-XXL


##### Q1. Find top 3 selling SKUs and bottom 3 selling SKUs in each category. Attach the SQL query used to extract this metric. (Exclude SKUs whose category data is missing) 


##### Answer :



#### Q Find total revenue generated corresponding to each category using following pricing data. Attach the Python/ R code used corresponding to above analysis.(Exclude SKUs whose category data is missing) 

In [71]:
price_df = pd.read_csv('price_range.csv')
price_df


,Category_id,Slab Range for orders,Price per order (Rs.)
0,"1,2",1-200,4.0
1,NaN,201-1000,2.0
2,NaN,Greater than 1000,1.0
3,"3,4",1-200,10.0
4,NaN,201-1000,7.5
5,NaN,Greater than 1000,5.0
6,All other category_id,1-200,7.0
7,NaN,201-1000,5.0
8,NaN,Greater than 1000,2.0


#### Devicing the function for calculating the revenue, thsi function will then mapped to the merged DF

In [72]:
category1 = [1,2]
category2 = [3,4]

def revenue_calc(category_id, order_count):
    if category_id in category1:
        if order_count in range(1,201):
            revenue  = order_count * 4 
        elif order_count in range(201,1001):
            revenue  = (order_count - 200) * 2 + 200*4
        else:
            revenue = (order_count-1000) * 1 + 800 * 2 + 200 * 4
    elif category_id in category2:
        if order_count in range(1,201):
            revenue  = order_count * 10 
        elif order_count in range(201,1001):
            revenue  = (order_count - 200) * 7.5 + 200*10
        else:
            revenue = (order_count-1000) * 5 + 800 * 7.5 + 200 * 10
    else:
        if order_count in range(1,201):
            revenue  = order_count * 7 
        elif order_count in range(201,1001):
            revenue  = (order_count - 200) * 7 + 200*7
        else:
            revenue = (order_count-1000) * 7 + 800 * 5 + 200 * 7
    return revenue
            

In [73]:
order_data.size

21700

In [74]:
category_data.size

31743

#### We will be merging the 2 dfs to get data at a level of category id and category count. Also we are using inner joins so that we have only that data where the sku_code and category mapping exists

In [75]:
df = pd.merge(order_data, category_data, how = 'inner', on = 'sku_code')
df.head()


,sku_code,order_count,category_id,category
0,805L,185,4,Fashion & Accessories
1,JR-202-38,32,4,Fashion & Accessories
2,K1786_L,69,4,Fashion & Accessories
3,FDRSS1744XL,6,4,Fashion & Accessories
4,BMAXBLUSQAR,56,5,FMCG & Agri


In [76]:
df.size


42612

In [77]:
df = df.drop_duplicates()

In [78]:
df.size

42324

In [79]:
df.groupby(['sku_code'])['sku_code'].count().sort_values()

sku_code
-8906102430009              1
JR-LK-Strip-432-Lemon-38    1
JR-LK-Strip-432-Lemon-40    1
JR-LK-Strip-432-Lemon-42    1
JR-LK-Strip-432-Lemon-44    1
                           ..
717M                        2
691XS                       2
192S                        3
192M                        3
FA4WH001                    3
Name: sku_code, Length: 10548, dtype: int64

In [80]:
order_data.sku_code.value_counts()

192S            3
192M            3
FA4WH001        3
717XS           2
746XL           2
               ..
726962607292    1
K1913_XXL       1
I-1Y46-ROLA     1
100002591       1
C3463           1
Name: sku_code, Length: 10816, dtype: int64

In [81]:
df.sku_code.value_counts()

192S              3
FA4WH001          3
192M              3
691S              2
691M              2
                 ..
RS-2160455        1
MV-3156228        1
LFS18028OGRN-L    1
FJCKT6149XL       1
C3463             1
Name: sku_code, Length: 10548, dtype: int64

In [82]:
df[df['sku_code'] == '192S']

,sku_code,order_count,category_id,category
3932,192S,14,4,Fashion & Accessories
3935,192S,5,4,Fashion & Accessories
3938,192S,7,4,Fashion & Accessories


#### Even after removing duplicates after the join we see there has been an explosion and this is due to the fact the initial order data df had multiple values of order_count for same sku, which is possible for different dates but date was not provided. We shall assume whatever is present in the data and go forward is calculating the revenue of each sku for each order

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10581 entries, 0 to 10652
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku_code     10581 non-null  object
 1   order_count  10581 non-null  int64 
 2   category_id  10581 non-null  int64 
 3   category     10581 non-null  object
dtypes: int64(2), object(2)
memory usage: 413.3+ KB


In [84]:
df.head()

,sku_code,order_count,category_id,category
0,805L,185,4,Fashion & Accessories
1,JR-202-38,32,4,Fashion & Accessories
2,K1786_L,69,4,Fashion & Accessories
3,FDRSS1744XL,6,4,Fashion & Accessories
4,BMAXBLUSQAR,56,5,FMCG & Agri


In [85]:
df['revenue'] = list(map(revenue_calc, df['category_id'], df['order_count']))


In [86]:
df.head()

,sku_code,order_count,category_id,category,revenue
0,805L,185,4,Fashion & Accessories,1850.0
1,JR-202-38,32,4,Fashion & Accessories,320.0
2,K1786_L,69,4,Fashion & Accessories,690.0
3,FDRSS1744XL,6,4,Fashion & Accessories,60.0
4,BMAXBLUSQAR,56,5,FMCG & Agri,392.0


In [87]:
df.sort_values(by = 'revenue', ascending = False)

,sku_code,order_count,category_id,category,revenue
4771,28269,20521,3,Eyewear & Accessories,105605.0
7344,0027888,16162,3,Eyewear & Accessories,83810.0
5597,35217,8249,3,Eyewear & Accessories,44245.0
1776,LNTR10S,4662,5,FMCG & Agri,31034.0
6504,LTRAY10,4662,5,FMCG & Agri,31034.0
...,...,...,...,...,...
3003,G1040,5,2,Electronic & Home Appliances,20.0
9975,L02547,5,2,Electronic & Home Appliances,20.0
6197,919231002075,5,1,Beauty & Wellness,20.0
7612,200100833,5,2,Electronic & Home Appliances,20.0


In [88]:
revenue_df = df.groupby(['category'])['revenue'].sum()

In [89]:
revenue_df = pd.DataFrame(revenue_df)
revenue_df.reset_index()

,category,revenue
0,Beauty & Wellness,166415.0
1,Electronic & Home Appliances,114053.0
2,Eyewear & Accessories,312492.5
3,FMCG & Agri,355676.0
4,Fashion & Accessories,2744302.5
5,Health & Pharma,130598.0


In [90]:
revenue_df.rename( columns  =  {'category':'Catgory_code','revenue':'Revenue(Rs.)'}, inplace = True)
revenue_df

,Revenue(Rs.)
category,
Beauty & Wellness,166415.0
Electronic & Home Appliances,114053.0
Eyewear & Accessories,312492.5
FMCG & Agri,355676.0
Fashion & Accessories,2744302.5
Health & Pharma,130598.0
